In [23]:
from tqdm import tqdm
import pandas as pd
import numpy as np 
import warnings

warnings.filterwarnings('ignore')

In [24]:
low = 6800
high = 6300 
med = 6200
hit = 5700

In [25]:
df = pd.read_csv(r'C:\Users\stpen\Downloads\Telegram Desktop\test.csv')

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245963 entries, 0 to 245962
Data columns (total 27 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   Код склада                                       245963 non-null  int64  
 1   ШК                                               245963 non-null  int64  
 2   Код товара                                       245963 non-null  object 
 3   Артикул поставщика                               245963 non-null  object 
 4   Внутренний артикул                               245963 non-null  object 
 5   ID Сайта                                         245963 non-null  int64  
 6   Товарное направление                             245963 non-null  object 
 7   Товарная группа                                  245963 non-null  object 
 8   Товарный кластер                                 245963 non-null  object 
 9   Проба          

In [27]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

data = df.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [28]:
df.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)', 'Дата закупки'], inplace=True)
data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'], inplace=True)
df = df.merge(data, how='outer', on='ID Сайта')

In [29]:
mean_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].mean()

In [34]:
mean_weight.columns = ['ID Сайта', 'Средний вес']

In [36]:
df = df.merge(mean_weight, how='outer', on='ID Сайта')

In [38]:
df['Ценовая корзина'] = np.where(
    (df['Дата закупки'] == 'Старые') & (df['Запрет скидки'] == 'Да') & (df['Скидка на изделии'] == 0), 'шок', 
    np.where(
        (df['Запрет скидки'] == 'Да') & (df['Скидка на изделии'] == 0), 'KVI', 
        np.where(
            df['Чистый вес'] < 1.5, 'дешевая',
            np.where(
                df['Чистый вес'] < 4, 'средняя', 'дорогая'
            )
        )
    )
)

In [41]:
df = df[['ШК', 'Товарная группа', 'ID Сайта', 'Артикул поставщика', 'Внутренний артикул', 'Проба', 'Ценовая корзина', 'Цена Розн., за шт', 'Чистый вес', 'Скидка на изделии', 'Запрет скидки', 'Дата закупки']]

In [42]:
df = df[df['Проба'] == 585]

In [43]:
df['Цена со скидками ДО'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии'] / 100) * 0.8 * 0.6
)

In [44]:
# result = pd.DataFrame({
#     'ШК': df['ШК'].tolist(),
#     'Вес': df['Чистый вес'].tolist(),
#     'Ценовая корзина': df['Ценовая корзина'].tolist(),
#     'Цена со скидками ДО': df['Цена со скидками ДО'].tolist(),
    
#                        })

In [45]:
df['Цена за грамм ПОСЛЕ'] = np.where(
    df['Ценовая корзина'] == 'дешевая', low, 
    np.where(
        df['Ценовая корзина'] == 'дорогая', high, 
        np.where(
            df['Ценовая корзина'] == 'средняя', med, hit
        )
    )
)

In [46]:
df['Скидка на изделии ПОСЛЕ'] = np.where(
    df['Ценовая корзина'] == 'дешевая', 60, 
    np.where(
        df['Ценовая корзина'] == 'дорогая', 40, 
        np.where(
            df['Ценовая корзина'] == 'средняя', 50, 0
        )
    )
)

In [47]:
def change(price):
    return price.replace(price[-2:], '90')

df['Цена со скидками ПОСЛЕ'] = df['Цена за грамм ПОСЛЕ'] * df['Чистый вес']
# df['Цена со скидками ПОСЛЕ'] = df['Цена со скидками ПОСЛЕ'].astype(int).astype(str)
# df['Цена со скидками ПОСЛЕ'] = df['Цена со скидками ПОСЛЕ'].apply(change).astype(int)

In [48]:
df['Цена изделия на бирке ПОСЛЕ'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена со скидками ПОСЛЕ'] / (1 - df['Скидка на изделии ПОСЛЕ'] / 100), df['Цена со скидками ПОСЛЕ'] / (1 - df['Скидка на изделии ПОСЛЕ'] / 100) / 0.8 / 0.6
).astype(int).astype(str)

In [49]:
df['Цена изделия на бирке ПОСЛЕ'] = df['Цена изделия на бирке ПОСЛЕ'].apply(change).astype(int)

In [50]:
df['Цена за грамм ДО'] = round(df['Цена со скидками ДО'] / df['Чистый вес'])

In [51]:
df['Изменения цена на бирке, %'] = df['Цена изделия на бирке ПОСЛЕ'] / df['Цена Розн., за шт'] - 1

In [52]:
df['Изменения цена за грамм, %'] = df['Цена за грамм ПОСЛЕ'] / df['Цена за грамм ДО'] - 1

In [53]:
df['Изменения цена со скидками, %'] = df['Цена со скидками ПОСЛЕ'] / df['Цена со скидками ДО'] - 1

In [54]:
df.columns

Index(['ШК', 'Товарная группа', 'ID Сайта', 'Артикул поставщика',
       'Внутренний артикул', 'Проба', 'Ценовая корзина', 'Цена Розн., за шт',
       'Чистый вес', 'Скидка на изделии', 'Запрет скидки', 'Дата закупки',
       'Цена со скидками ДО', 'Цена за грамм ПОСЛЕ', 'Скидка на изделии ПОСЛЕ',
       'Цена со скидками ПОСЛЕ', 'Цена изделия на бирке ПОСЛЕ',
       'Цена за грамм ДО', 'Изменения цена на бирке, %',
       'Изменения цена за грамм, %', 'Изменения цена со скидками, %'],
      dtype='object')

In [55]:
df.columns = ['ШК', 'ID Сайта', 'Артикул поставщика',
       'Внутренний артикул', 'Товарная группа', 'Проба', 'Ценовая корзина',
       'Цена на бирке ДО', 'Чистый вес', 'Скидка на изделии ДО', 'Запрет скидки',
       'Дата закупки', 'Цена со скидками ДО', 'Цена за грамм ПОСЛЕ',
       'Скидка на изделии ПОСЛЕ', 'Цена со скидками ПОСЛЕ', 'Цена изделия на бирке ПОСЛЕ',
       'Цена за грамм ДО', 'Изменения цена на бирке, %',
       'Изменения цена за грамм, %', 'Изменения цена со скидками, %']

In [56]:
df

,ШК,ID Сайта,Артикул поставщика,Внутренний артикул,Товарная группа,Проба,Ценовая корзина,Цена на бирке ДО,Чистый вес,Скидка на изделии ДО,...,Дата закупки,Цена со скидками ДО,Цена за грамм ПОСЛЕ,Скидка на изделии ПОСЛЕ,Цена со скидками ПОСЛЕ,Цена изделия на бирке ПОСЛЕ,Цена за грамм ДО,"Изменения цена на бирке, %","Изменения цена за грамм, %","Изменения цена со скидками, %"
0,2078477989607,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,дорогая,100194.0,5.17,34.0,...,Старые,31741.4592,6300,40,32571.0,113090,6140.0,0.128710,0.026059,0.026134
1,2078477988518,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,дорогая,90717.0,4.65,34.0,...,Старые,28739.1456,6300,40,29295.0,101790,6180.0,0.122061,0.019417,0.019341
2,2078477985623,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,дорогая,104542.0,5.38,39.0,...,Старые,30609.8976,6300,40,33894.0,117690,5690.0,0.125768,0.107206,0.107289
3,2078477985548,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,дорогая,104153.0,5.36,39.0,...,Старые,30495.9984,6300,40,33768.0,117290,5690.0,0.126132,0.107206,0.107293
4,2078477985692,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,дорогая,104454.0,5.34,39.0,...,Старые,30584.1312,6300,40,33642.0,116890,5727.0,0.119057,0.100052,0.099982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245958,2078113911672,БК КОЛЬЦА ОБРУЧ,2018099,11010746,11010746_БК,585,шок,9360.0,2.08,0.0,...,Старые,9360.0000,5700,0,11856.0,11890,4500.0,0.270299,0.266667,0.266667
245959,2078143278998,БК КОЛЬЦА ОБРУЧ,2017058,А-30-01-2-09-049,А-30-01-2-09-049_БК,585,шок,5031.0,1.28,0.0,...,Старые,5031.0000,5700,0,7296.0,7290,3930.0,0.449016,0.450382,0.450209
245960,2078071885985,БК КОЛЬЦА ОБРУЧ,2010672,305025,305025_БК,585,шок,8370.0,1.86,0.0,...,Старые,8370.0000,5700,0,10602.0,10690,4500.0,0.277180,0.266667,0.266667
245961,2078156841028,БК КОЛЬЦА ОБРУЧ,2002047,ГБ-50-01-2-13-000,ГБ-50-01-2-13-000_БК,585,шок,7695.0,1.71,0.0,...,Старые,7695.0000,5700,0,9747.0,9790,4500.0,0.272255,0.266667,0.266667


In [57]:
df = df[['ШК', 'ID Сайта', 'Артикул поставщика',
       'Внутренний артикул', 'Товарная группа', 'Проба', 'Ценовая корзина',
       'Цена на бирке ДО', 'Чистый вес', 'Скидка на изделии ДО', 'Запрет скидки',
       'Цена со скидками ДО', 'Цена за грамм ДО',
       'Цена за грамм ПОСЛЕ', 'Скидка на изделии ПОСЛЕ',
       'Цена со скидками ПОСЛЕ', 'Цена изделия на бирке ПОСЛЕ',
       'Изменения цена на бирке, %',
       'Изменения цена за грамм, %', 'Изменения цена со скидками, %']]

In [58]:
df.dropna(inplace=True)

In [59]:
df['Изменения цена на бирке, %'] = np.where(
    df['Изменения цена на бирке, %'] < 0, df['Изменения цена на бирке, %'] * -1, df['Изменения цена на бирке, %']
)

In [60]:
df['Изменения цена за грамм, %'] = np.where(
    df['Изменения цена за грамм, %'] < 0, df['Изменения цена за грамм, %'] * -1, df['Изменения цена за грамм, %']
)

In [61]:
df['Замена бирки'] = np.where(
    df['Изменения цена на бирке, %'] > 0.05, True, False
)

In [62]:
df['Переоценка'] = np.where(
    df['Изменения цена за грамм, %'] > 0.05, True, False
) 

In [63]:
result_df = df

In [64]:
result_df.to_excel('изменения по БК КОЛЬЦА ОБРУЧ.xlsx', index=False)

In [229]:
df = pd.read_csv(r'C:\Users\stpen\Downloads\Telegram Desktop\test.csv')

In [230]:
df = df[['ШК', 'Товарная группа', 'ID Сайта', 'Артикул поставщика', 'Внутренний артикул', 'Проба', 'Ценовая корзина', 'Цена Розн., за шт', 'Чистый вес', 'Скидка на изделии', 'Запрет скидки', 'Дата закупки']]

In [231]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

data = df.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [232]:
df.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)', 'Дата закупки'], inplace=True)
data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'], inplace=True)
df = df.merge(data, how='outer', on='ID Сайта')

In [233]:
df['Ценовая корзина'] = np.where(
    df['Дата закупки'] == 'Старые', 'шок', 
    np.where(
        (df['Запрет скидки'] == 'Да') & (df['Скидка на изделии'] == 0), 'KVI', 
        np.where(
            df['Чистый вес'] < 1.5, 'дешевая',
            np.where(
                df['Чистый вес'] < 4, 'средняя', 'дорогая'
            )
        )
    )
)

In [234]:
df['Цена за грамм ПОСЛЕ'] = np.where(
    df['Ценовая корзина'] == 'дешевая', low, 
    np.where(
        df['Ценовая корзина'] == 'дорогая', high, 
        np.where(
            df['Ценовая корзина'] == 'средняя', med, hit
        )
    )
)

In [247]:
df

,ШК,Товарная группа,ID Сайта,Артикул поставщика,Внутренний артикул,Проба,Ценовая корзина,"Цена Розн., за шт",Чистый вес,Скидка на изделии,Запрет скидки,Дата закупки,Цена за грамм ПОСЛЕ,Скидка на изделии НОВАЯ,Цена со скидками НОВАЯ,Цена без базовой скидки
0,2078477989607,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,шок,100194.0,5.17,34.0,Нет,Старые,5700,0.387251,29469.0,61393.75
1,2078477988518,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,шок,90717.0,4.65,34.0,Нет,Старые,5700,0.391308,26505.0,55218.75
2,2078477985623,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,шок,104542.0,5.38,39.0,Нет,Старые,5700,0.388882,30666.0,63887.50
3,2078477985548,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,шок,104153.0,5.36,39.0,Нет,Старые,5700,0.388880,30552.0,63650.00
4,2078477985692,БК КОЛЬЦА ОБРУЧ,2246278,0102160674-50000,0102160674-50000_БК,585,шок,104454.0,5.34,39.0,Нет,Старые,5700,0.392915,30438.0,63412.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245958,2078113911672,БК КОЛЬЦА ОБРУЧ,2018099,11010746,11010746_БК,585,шок,9360.0,2.08,0.0,Да,Старые,5700,NaN,NaN,NaN
245959,2078143278998,БК КОЛЬЦА ОБРУЧ,2017058,А-30-01-2-09-049,А-30-01-2-09-049_БК,585,шок,5031.0,1.28,0.0,Да,Старые,5700,NaN,NaN,NaN
245960,2078071885985,БК КОЛЬЦА ОБРУЧ,2010672,305025,305025_БК,585,шок,8370.0,1.86,0.0,Да,Старые,5700,NaN,NaN,NaN
245961,2078156841028,БК КОЛЬЦА ОБРУЧ,2002047,ГБ-50-01-2-13-000,ГБ-50-01-2-13-000_БК,585,шок,7695.0,1.71,0.0,Да,Старые,5700,NaN,NaN,NaN


In [246]:
df['Скидка на изделии НОВАЯ'] = np.nan
df['Цена со скидками НОВАЯ'] = np.nan
df['Цена без базовой скидки'] = np.nan

for i in df.index: 
    if df['Запрет скидки'][i] == 'Нет':
        df['Цена со скидками НОВАЯ'][i] = df['Цена за грамм ПОСЛЕ'][i] * df['Чистый вес'][i]
        df['Цена без базовой скидки'][i] = df['Цена со скидками НОВАЯ'][i] / 0.8 / 0.6
        df['Скидка на изделии НОВАЯ'][i] = 1 - df['Цена без базовой скидки'][i] / df['Цена Розн., за шт'][i]     

In [ ]:
df['Цена изделия на бирке ПОСЛЕ'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена со скидками ПОСЛЕ'] / (1 - df['Скидка на изделии ПОСЛЕ'] / 100), df['Цена со скидками ПОСЛЕ'] / (1 - df['Скидка на изделии ПОСЛЕ'] / 100) / 0.8 / 0.6
).astype(int).astype(str)